In [1]:
import requests
import json
import pandas as pd

In [2]:
# NHS Terminology server sign-in details
client_id = "***"
client_secret = "****"

#API endpoints
authoring_server = "https://ontology.nhs.uk/authoring/fhir"
production1_server = "https://ontology.nhs.uk/production1/fhir"
production2_server = "https://ontology.nhs.uk/production2/fhir"
token_server = "https://ontology.nhs.uk/authorisation/auth/realms/nhs-digital-terminology/protocol/openid-connect/token"
authorisation_server = "https://ontology.nhs.uk/authorisation/auth/realms/nhs-digital-terminology/protocol/openid-connect/auth"

# get access_token from token server
auth_data =  {'grant_type': 'client_credentials', 'client_id': client_id, 'client_secret': client_secret} 
token_response_json = requests.post(token_server, data=auth_data).json()
access_token=token_response_json['access_token']
expires_in=token_response_json['expires_in'] # could be used to time new token request or refresh
refresh_token=token_response_json['refresh_token'] # can be used to refresh access_token


In [3]:
## function for pulling the ECDS (Emergency care data set) 
def RefList(RefsetId):
    lookup_url=production1_server + '/ValueSet/$expand'
    request_headers = {'Authorization': 'Bearer ' + access_token}
    query={'url':'http://snomed.info/sct?fhir_vs=refset/' + RefsetId, 'count':1500}
    lookup_response_json = requests.get(lookup_url, headers=request_headers, params=query).json()

    _list = []
    _list2 = []
    for a in lookup_response_json['expansion']['contains']:
        _list.append(a['code'])
        _list2.append(a['display'])
                    
    #convert to a pandas dataframe and rename column headers
    dataset = pd.DataFrame(list(zip(_list, _list2)),
                  columns =['code', 'display'])
    
    return dataset


In [40]:
ECDS_diag = RefList('991411000000109') # diagnosis code list
ECDS_CC = RefList('991401000000107') # chief complaints code list

In [3]:
## Expression Constraint Language as function
def ECL_to_conceptid(ecl):
    lookup_url=production1_server + '/ValueSet/$expand'
    request_headers = {'Authorization': 'Bearer ' + access_token}
    query={'url':'http%3A%2F%2Fsnomed.info%2Fsct%3Ffhir_vs%3Decl%2F'+ ecl, 'count':1000}
    lookup_response_json = requests.get(lookup_url, headers=request_headers, params=query).json()
    _list = []
    _list2 = []
    for a in lookup_response_json['expansion']['contains']:
        _list.append(a['code'])
        _list2.append(a['display'])

        
    #convert to a pandas dataframe and rename column headers
    dataset = pd.DataFrame(list(zip(_list, _list2)),
                  columns =['code', 'display'])
    
    return dataset

# Vaccines list

In [5]:
vaccine = ECL_to_conceptid('<<39330711000001103 OR (<<10363601000001109:<<10362601000001103= <<39330711000001103)')
#vaccine.to_csv('v.csv')

In [31]:
platelets = ECL_to_conceptid('<<22716005:<< 363713009 = << 281300000,<< 363714003 = << 61928009')
thrombosis =  ECL_to_conceptid('<<64572001 : 116676008 = << 75753009')
bloodclots = ECL_to_conceptid('<< 75753009')

##  can thrombotic events be coded in ED datasets?

In [44]:
platelets[platelets.code.isin(ECDS_CC['code'])]

,code,display


In [43]:
thrombosis[thrombosis.code.isin(ECDS_diag['code'])]


,code,display
50,266262004,Arterial embolus and thrombosis
83,68478007,Central retinal vein occlusion
139,128053003,Deep venous thrombosis
188,297157005,Intracranial venous thrombosis
326,40283005,Thrombophlebitis of superficial veins of lower...
459,24596005,Venous retinal branch occlusion


In [24]:
ECDS_diag # full ED diagnosis code list

,code,display
0,233985008,Abdominal aortic aneurysm
1,398925009,Abducens nerve disorder
2,274127000,Abnormal delivery
3,110167007,Abrasion of circumoral region of face
4,283051009,Abrasion of ear region
...,...,...
1181,276191000000107,Viral wheeze
1182,15013002,Vitreous floaters
1183,822061000000106,West Nile fever
1184,70647001,Yaws


In [21]:
platelets # full platelets code list

,code,display
0,55907008,Acquired aplastic anaemia
1,5876000,Acquired pancytopenia
2,234376007,Acquired red cell aplasia
3,74576004,Acquired thrombocytopenia
4,439007008,Acquired thrombotic thrombocytopenic purpura
...,...,...
136,416902009,Uraemic thrombocytopenia
137,403837005,Wiskott-Aldrich autosomal dominant variant syn...
138,36070007,Wiskott-Aldrich syndrome
139,719019000,WT limb blood syndrome
